### Environment Config

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

### Read Dataset

Data was preprocessed by other script available in: https://github.com/HWatanuki/Projeto_D3TOP/blob/main/Codigos/Data_preprocessing_v0.3.ipynb

In [2]:
file_path = 'https://media.githubusercontent.com/media/HWatanuki/Projeto_D3TOP/main/Datasets/processed/'
file_name = 'dataset_airbnb-processed-spacy_2023-04-13_03-28-09-439.csv'

df = pd.read_csv(file_path + file_name, dtype=str)
print(f'No Rows: {df.shape[0]}')
print(f'No Columns: {df.shape[1]}', end='\n\n')
display(df.head())

No Rows: 561
No Columns: 94



,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/listingsCount,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,primaryHost/totalListingsCount,reviews/0/author/firstName,reviews/0/author/id,reviews/0/createdAt,reviews/0/id,reviews/0/rating,reviews/1/author/firstName,reviews/1/author/id,reviews/1/createdAt,reviews/1/id,reviews/1/rating,reviews/2/author/firstName,reviews/2/author/id,reviews/2/createdAt,reviews/2/id,reviews/2/rating,reviews/3/author/firstName,reviews/3/author/id,reviews/3/createdAt,reviews/3/id,reviews/3/rating,reviews/4/author/firstName,reviews/4/author/id,reviews/4/createdAt,reviews/4/id,reviews/4/rating,reviews/5/author/firstName,reviews/5/author/id,reviews/5/createdAt,reviews/5/id,reviews/5/rating,reviews/6/author/firstName,reviews/6/author/id,reviews/6/createdAt,reviews/6/id,reviews/6/rating,reviews/7/author/firstName,reviews/7/author/id,reviews/7/createdAt,reviews/7/id,reviews/7/rating,reviews/8/author/firstName,reviews/8/author/id,reviews/8/createdAt,reviews/8/id,reviews/8/rating,reviews/9/author/firstName,reviews/9/author/id,reviews/9/createdAt,reviews/9/id,reviews/9/rating,roomType,stars,url,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments
0,"Jersey City, New Jersey, United States",true,false,40.7233,-74.03946,2,NaN,https://a0.muscache.com/im/pictures/miso/Hosting-53775685/original/0812b9a1-2d2e-4725-b51f-881045a939b2.jpeg?aki_policy=large,https://a0.muscache.com/im/pictures/miso/Hosting-53775685/original/0812b9a1-2d2e-4725-b51f-881045a939b2.jpeg?aki_policy=small,99999,"$99,999",USD,false,nightly,82 Reviews,Identity verified,NaN,Joes,false,true,245395267,false,中文 (简体),8,Joined in February 2019,NaN,NaN,Joes,14,Miranda,74889676.0,2022-06-20T19:58:54Z,6.536263641559459e+17,4,Abhimanyu,4800160.0,2022-05-23T18:57:26Z,6.333017075212511e+17,5,Robert,162871850.0,2022-05-06T17:28:10Z,6.209355874005248e+17,5,Yu,303996502.0,2022-01-09T21:55:09Z,5.362712073993113e+17,5,Mitesh,253080011.0,2021-12-26T20:27:02Z,5.260799932553211e+17,4.0,Arusha,103758126.0,2021-12-22T21:29:10Z,5.2321216515417446e+17,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entire rental unit,4.67,https://www.airbnb.com/rooms/53775685,entire cozy unit mins manhattan,hello welcome love see world able help people feel right home,great location nice building love parking include,stay perfect mention listing location min walk newport path right opposite huge grocery store joes super kind early checkin late checkout,excellent place stay central fo travel nyc work travel new jersey stay months apartment place advertise joes excellent host,best airbnb stay get great view outside window comfortable convenient living experience settle great,nice apartment great locality apartment list definitely rate star airbnb pet hair carpet couch month old bit concerned have allergies look clean place joes approachable easy communicate,customer take chance trust listing apartment exceed expectations super clean aesthetically pleasing minimalistic vibe definitely luxury apartment building convenient location right shoprite near waterfront path host responsive questions thanks great stay,NaN,NaN,NaN,NaN
1,"New York, United States",true,false,40.70641,-74.0092,2,NaN,https://a0.muscache.com/im/pictures/e2388507-1f5f-4000-aa1b-d3b2279e682a.jpg?aki_policy=large,https://a0.muscache.com/im/pictures/e2388507-1f5f-4000

### Data preparation

In [12]:
# Join all reviews comments for each announcement record
df_s01 = df.copy()
df_s01['comments'] = df_s01[df_s01.columns[df_s01.columns.str.contains('comments')]].agg(lambda x: '  '.join([str(c) for c in x.values if str(c) != 'nan']), axis=1).T

In [13]:
df_s02 = df_s01[['address', 'numberOfGuests', 'name', 'pricing/rate/amount', 'roomType', 'stars',  'comments']].copy()
df_s02.sample(2)

,address,numberOfGuests,name,pricing/rate/amount,roomType,stars,comments
503,"New York, United States",4,epic location spacious br loft midtown west,400,Entire rental unit,4.59,apartment great hell kitchen location brilliant restaurants bars nearby try pocket bar backpocket bar round corner facilities need little roof terrace drink sunny night wonderful views manhattan hosts responsive arrangements simple communicate june great host responsivegreat location blocks broadway time square rockefeller centre attractionsapartment clean spacious great pizza place groceries apartmentwould love come trip nyc stay junes airbnb amazing airbnb comfortable spacious locate delis restaurants subway stations comfortably fit people bathroom inconvenient overall excellent apartment stay june responsive need actual address corner address provide expectedplace description photos change newer sofa bed lounge roomeasy walk circle ferry broadway times square approximately min walk central park great location hells kitchenunit th floor stairs entrance level level love location bar right noisy late night bother best renters aware leave bedroom doors open get hot stuffy problem share plenty space comfortable beds clean good location ushost readily available need communication great spot nyc weekend enjoyable stay apartment clean comfortable good area hells kitchen walkable lot restaurants coffee shops convenient markets door skyline deli great build bit old elevator bit hike th floor luggage wasn issue june responsive accommodating definitely stay th floor narrow stairs elevator faint heart health challenge door code keys set checkin personally preplanned meeting time st cohost bring hand towels soap bath dishwasher coffee salt pepper linens pillows blankets wear luke warm showers quiet building kitchen clean new appliances plenty pots pans dishes cook location walkable shows times square rockefeller center area interesting restaurants bars close unaware deck rooftop area instructions apartment details cool nycstyle apartment amazing area great apartment outdoor terrace amazing views great time appreciate fast replies door hard open close fix blink thank june perfect place group location great easy access want family accomplish
483,"New York, United States",2,sunny fullfloor loft downtown nyc,1000,Entire loft,5.0,apartment amazing look like film set bathtub steam shower projector triple exposure bridge skyline view vintage tin ceilings plants cloudlike bed completely dreamy yoga studio upstairs great nico loft massive nyc sunlight plants truly oasis busy downtown loft need cook laundry bed probably comfiest see airbnb shower steam room combo cherry love special place yoga studio amazing steam room best super cool good communication owners great stay place refuge stay perfect enjoy stay apartment beautifully set check check appreciate efficient air conditioning hot summer weeks amenities great huge space plants place feel like relaxing haven hectic city hope soon great special experience yoga studio upstairs extra bonus


### TF-IDF for get relevant words

In [14]:
max_words = 10000
vectoriser = TfidfVectorizer(max_features=max_words)
tfidf_result = vectoriser.fit_transform(df_s02['comments'])
feature_names = vectoriser.get_feature_names_out()
print(f'Data Transformed.')
print('No. of feature_words: ', len(feature_names))

Data Transformed.
No. of feature_words:  6968


In [43]:
def get_summary(row):
  '''
  Function that iterates through the comment (all reviews together), calculating a score for each sentence by summing the TF-IDF values for each word appearing in the sentence. 
  Normalize the sentence scores by dividing by the number of tokens in the sentence (to avoid bias in favor of longer sentences). 
  Then sort the sentences by their scores, and return the highest-scoring sentences as the summary. 
  The number of sentences returned corresponds to roughly 20% of the overall length of the comment.
  '''
  import math

  comment_id = row.name  
  comment_text = row['comments']  
  sent_scores = []
  if not len(comment_text):
    return ''

  for sentence in comment_text.split('  '):    
    score = 0    
    sent_tokens = sentence.split()
    if not len(sent_tokens):
      continue
    for token in (t for t in sent_tokens if t in feature_names):
        score += tfidf_result[comment_id, np.squeeze(np.where(feature_names == token))]      
    sent_scores.append((score / len(sent_tokens), sentence))

  summary_length = int(math.ceil(len(sent_scores) / 5))
  sent_scores.sort(key=lambda sent: sent[0], reverse=True)

  summary_parts = []
  for summary_sentence in sent_scores[:summary_length]:
    summary_parts.append(summary_sentence[1])
  return ' '.join(summary_parts)



#### Create a new variable containing more relevant words of reviews as the summary

In [44]:
%%time
df_s02['relevant_words'] = df_s02.apply(lambda x: get_summary(x) , axis=1)
df_s02.sample(2)

CPU times: user 32.8 s, sys: 30.7 ms, total: 32.9 s
Wall time: 33 s


,address,numberOfGuests,name,pricing/rate/amount,roomType,stars,comments,relevant_words
241,"New York, United States",8,columbus circle central park theater district,1095,Entire rental unit,NaN,decide want rent bedroom nyc month july wife want space collegeaged children probably little late start rental search fewer options time look rick list apartment pop review space photos rick list superhost decide chance unreviewed listing glad let start pluses dominate apartment spacious comfortable decor simple minimalist functional beds living room couch comfortable sleep enjoy large screen tv live roomthere decent storage clothes large walkin hallway closet provide plenty additional space bathrooms small adequate choose bedrooms attached bath bigger brighter pluses space impeccably clean restaurant ground level building air condition hot july extremely quiet rooms face north suoer quiet bedroom face south street noise isn terribly noisy area location simply perfect plus caretaker alexis incredibly responsive questions need check couldn pleased ll include negatives weren negatives th floor walk second kitchen definitely small finally pricey want bed bath expensive area happy choice sorry leave month come end objective explore city clean comfortable conveniently locate apartment base happy,decide want rent bedroom nyc month july wife want space collegeaged children probably little late start rental search fewer options time look rick list apartment pop review space photos rick list superhost decide chance unreviewed listing glad let start pluses dominate apartment spacious comfortable decor simple minimalist functional beds living room couch comfortable sleep enjoy large screen tv live roomthere decent storage clothes large walkin hallway closet provide plenty additional space bathrooms small adequate choose bedrooms attached bath bigger brighter pluses space impeccably clean restaurant ground level building air condition hot july extremely quiet rooms face north suoer quiet bedroom face south street noise isn terribly noisy area location simply perfect plus caretaker alexis incredibly responsive questions need check couldn pleased ll include negatives weren negatives th floor walk second kitchen definitely small finally pricey want bed bath expensive area happy choice sorry leave month come end objective explore city clean comfortable conveniently locate apartment base happy
104,"New York, United States",4,stunning tribeca loft chefs kitchen deck,795,Entire condo,5.0,great things luke place spacious clean beds super comfortable travel month old need luke responsive questions need stay location awesome definitely goto anytime town amazing place stay perfect place young family young ones toys open space life saver kiddos entertain inside apartment highly recommend place luke wife assistant actively communicate easy reach wait come wow beautiful apartment fantastic amenities beds incredibly comfortable location great hosts check happily stay place problems think leftover watermelon showers storage amazing place thank have perfect place stay ny family long weekend luke crystal excellent hosts responsive easy checkin location perfect close train restaurants quiet neighborhood feel safe definitely stay beautiful apt good layout true photos large comfortable bedrooms dark shades perfect sleep near restaurants play ground hudson river park right corner friendly responsive owners easy communicate definitely rebook time city,amazing place stay perfect place young family young ones toys open space life saver kiddos entertain inside apartment highly recommend place luke wife assistant actively communicate easy reach wait come


## Recommendation system

In [45]:
dfs_03 = df_s02.copy()
dfs_03.fillna('', inplace=True)
dfs_03.head(1)

,address,numberOfGuests,name,pricing/rate/amount,roomType,stars,comments,relevant_words
0,"Jersey City, New Jersey, United States",2,entire cozy unit mins manhattan,99999,Entire rental unit,4.67,great location nice building love parking include stay perfect mention listing location min walk newport path right opposite huge grocery store joes super kind early checkin late checkout excellent place stay central fo travel nyc work travel new jersey stay months apartment place advertise joes excellent host best airbnb stay get great view outside window comfortable convenient living experience settle great nice apartment great locality apartment list definitely rate star airbnb pet hair carpet couch month old bit concerned have allergies look clean place joes approachable easy communicate customer take chance trust listing apartment exceed expectations super clean aesthetically pleasing minimalistic vibe definitely luxury apartment building convenient location right shoprite near waterfront path host responsive questions thanks great stay,travel new jersey stay months apartment place advertise joes excellent host nice apartment great locality apartment list definitely rate star airbnb pet hair carpet couch month old bit concerned have allergies look clean place joes approachable easy communicate


In [46]:
# TF-IDF
max_words = 10000
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), max_features=max_words)
tfidf_matrix = vectorizer.fit_transform(dfs_03['comments'] + "" + dfs_03['relevant_words'])
tfidf_feature_names = vectorizer.get_feature_names_out()

In [47]:
input_search = str(input())
df_input = pd.DataFrame(np.array([[input_search]]))
pp_input = vectorizer.transform(df_input[0])

have microwave


In [50]:
# Computing cosine similarity of user input
cos_similarity_tfidf = map(lambda x: cosine_similarity(pp_input, x), tfidf_matrix)

In [51]:
simi_scores = list(cos_similarity_tfidf)
simi_scores = [(i, score.squeeze(0)[0]) for i, score in enumerate(simi_scores)]
simi_scores = sorted(simi_scores, key=lambda x: x[1], reverse=True)

In [52]:
announces_index =[i[0] for i in simi_scores]

In [54]:
top_n = 5
dfs_03.iloc[announces_index][:top_n]

,address,numberOfGuests,name,pricing/rate/amount,roomType,stars,comments,relevant_words
163,"Clifton, New Jersey, United States",2,spacious private room near nyc,4554,Private room in home,4.9,ysabel amazing host help settle home responsive agreeable minute changes couldn recommend place extremely affordable spotless private locks door bedroom extra security lock outside bedroom need feel comfortable leave valuables friendly host accommodating sweet EMOJIsmile excellent spotless comfortable need ysabel accommodate kind great stay recommend nd stay calm comfortable place nice definitely great place private basement room mini living room come microwave mini fridge room clean bathroom pluswould nice desk think wifi bit weaker basement awesome place nice idiot nt book cleanest airbnb stay travel year star ysabel incredibly kind informative location ideal,excellent spotless comfortable need ysabel accommodate kind ysabel incredibly kind informative location ideal great place private basement room mini living room come microwave mini fridge room clean bathroom pluswould nice desk think wifi bit weaker basement awesome place
391,"New York, United States",4,guest bedroom double beds courtyard new york manhattan fifth avenue,499,Entire rental unit,4.58,great location staff super friendly room small didn expect nyc recommend small fridge microwave complaints definitely stay amazing experience hotel staff complete stay great people unbeatable location clean room accommodating definitely stay wonderful stay nyc location fantastic check smooth easy hotel staff nice helpful provide great restaurant recommendations area absolutely recommend hotel perfect room big didn hear noise location perfect lots stores restaurants near love place coffee water check check easy check prices book cool place good stay stay heredo book airbnb charge overall total tax sure screenshot fees finalize pay tax airbnb tax pure air quality hotel hold day want tell release business days charge gold night second night hold day charge check hold new york expensive air quality horrible ducts need clean dust come vent say apartment don fridge store stuff microwave warm room small price pay stay nice place like recommend friends stay couple nights visit nyc best location room small fridge parking,stay heredo book airbnb charge best location room small fridge parking overall total tax sure screenshot fees finalize pay tax airbnb tax pure air quality hotel hold day want tell release business days charge gold night second night hold day charge check hold new york expensive air quality horrible ducts need clean dust come vent say apartment don fridge store stuff microwave warm room small price pay stay
378,"New York, United States",2,hotel room king bed chelsea,582,Room in hotel,4.8,great place stay super clean attentive staff hotel great rooms great price location middle flower district like plants rooftop absolutely gorgeous night staff welcome accommodate patient airline lose bags staff accommodating help situation nice hotel chelsea good value pay great location friendly staff clean update accommodations highly recommend room neat clean comfortable picture definitely stay location perfect koreatown manhattan subways literally right distance check simple michelle help parking car park days street minute walk concern blanket hole appear stain stay good amazingly microwave stay issue solve easily ask blanket wellrun place clean quiet nicely locate head manhattan especially downtown comfy bed happily reserve need city michelle place excellentchoice work pleasure love roof terrace room layout great location right near penn station affordable location close walking distance afew good restaurants public transportation shops excelente ubicacion limpieza todos muy amables,excelente ubicacion limpieza todos muy amables great place stay super clean attentive staff
59,"New York, United States",4,large luxury bedrooms apartment th st,2000,Entire serviced apartment,4.5,relocate ny stay 